### Notebook for concept detection in neural network

In [2]:
%load_ext autoreload
%autoreload 2

In [32]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

from concepts import static_concepts, linear_regression
import env
from policy import ConvNet

model_name = "net"
session_name = "blueorigin"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.play_center_in_opening

CONCEPT_NAME = static_concepts.play_center_in_opening.__name__

CASES_TO_SAMPLE = 2500 # 25000


In [33]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [35]:
def play_match(agents: list[ConvNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    go_env.reset()

    state = go_env.canonical_state()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.8
    alpha = 1

    game_over = False

    while not game_over:
        if moves == total_moves:
            break

        valid_moves = go_env.valid_moves()
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [2,3,4,5], axis=0)
        
        if np.random.random() < sample_ratio:
            pos = concept_function(state)
            if pos:
                positive_cases.append(state_copy)
            elif not pos:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state_copy, valid_moves, alpha=alpha)

        _, _, game_over, _ = go_env.step(action)
        state = go_env.canonical_state()
        moves += 1

        current_player = 1 - current_player
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)

            positive_cases.extend(pos)
            negative_cases.extend(neg)

            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   4%|▍         | 106/2500 [00:05<02:11, 18.20it/s]
Positive cases: 2600it [01:09, 44.13it/s]                          

Positive cases: 2600it [01:20, 44.13it/s]

In [36]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 2, 5, 5)
Negative cases:  (2500, 2, 5, 5)


In [37]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [38]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  2317
Duplicate count in negative:  1312


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [40]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [41]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.8162 - val_loss: 0.8162
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7942 - val_loss: 0.7943
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7750 - val_loss: 0.7754
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7582 - val_loss: 0.7584
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7431 - val_loss: 0.7432
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7293 - val_loss: 0.7290
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7166 - val_loss: 0.7159
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7049 - val_loss: 0.7038
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6941 - val_loss: 0.6928
Epoch 10/50
125/125 [==============================] - 0s 2ms/step - loss: 0.6842 - val_loss: 0.6824

In [42]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)


Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 308.92it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9314 - val_loss: 0.9049
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8818 - val_loss: 0.8581
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8380 - val_loss: 0.8170
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7995 - val_loss: 0.7815
Epoch 5/50
125/125 [==============================] - 0s 957us/step - loss: 0.7676 - val_loss: 0.7529
Epoch 6/50
125/125 [==============================] - 0s 992us/step - loss: 0.7414 - val_loss: 0.7289
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7199 - val_loss: 0.7104
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7042 - val_loss: 0.6974
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6941 - val_loss: 0.6908
Epoch 10/50
125/125 [==============================] - 0s 1ms/

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 339.55it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9167 - val_loss: 0.8809
Epoch 2/50
125/125 [==============================] - 0s 987us/step - loss: 0.8652 - val_loss: 0.8328
Epoch 3/50
125/125 [==============================] - 0s 947us/step - loss: 0.8215 - val_loss: 0.7919
Epoch 4/50
125/125 [==============================] - 0s 928us/step - loss: 0.7840 - val_loss: 0.7572
Epoch 5/50
125/125 [==============================] - 0s 938us/step - loss: 0.7520 - val_loss: 0.7279
Epoch 6/50
125/125 [==============================] - 0s 945us/step - loss: 0.7256 - val_loss: 0.7038
Epoch 7/50
125/125 [==============================] - 0s 949us/step - loss: 0.7036 - val_loss: 0.6840
Epoch 8/50
125/125 [==============================] - 0s 933us/step - loss: 0.6864 - val_loss: 0.6697
Epoch 9/50
125/125 [==============================] - 0s 936us/step - loss: 0.6746 - val_loss: 0.6610
Epoch 10/50
125/125 [=============================

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 309.19it/s]

...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9142 - val_loss: 0.8787
Epoch 2/50
125/125 [==============================] - 0s 972us/step - loss: 0.8541 - val_loss: 0.8235
Epoch 3/50
125/125 [==============================] - 0s 936us/step - loss: 0.8033 - val_loss: 0.7762
Epoch 4/50
125/125 [==============================] - 0s 933us/step - loss: 0.7595 - val_loss: 0.7358
Epoch 5/50
125/125 [==============================] - 0s 926us/step - loss: 0.7217 - val_loss: 0.7005
Epoch 6/50
125/125 [==============================] - 0s 927us/step - loss: 0.6893 - val_loss: 0.6712
Epoch 7/50
125/125 [==============================] - 0s 935us/step - loss: 0.6625 - val_loss: 0.6470
Epoch 8/50
125/125 [==============================] - 0s 918us/step - loss: 0.6411 - val_loss: 0.6287
Epoch 9/50
125/125 [==============================] - 0s 915us/step - loss: 0.6252 - val_loss: 0.6150
Epoch 10/50
125/125 [=============================

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 342.17it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9461 - val_loss: 0.9183
Epoch 2/50
125/125 [==============================] - 0s 990us/step - loss: 0.8854 - val_loss: 0.8631
Epoch 3/50
125/125 [==============================] - 0s 932us/step - loss: 0.8322 - val_loss: 0.8145
Epoch 4/50
125/125 [==============================] - 0s 935us/step - loss: 0.7856 - val_loss: 0.7714
Epoch 5/50
125/125 [==============================] - 0s 918us/step - loss: 0.7442 - val_loss: 0.7335
Epoch 6/50
125/125 [==============================] - 0s 920us/step - loss: 0.7089 - val_loss: 0.7019
Epoch 7/50
125/125 [==============================] - 0s 921us/step - loss: 0.6801 - val_loss: 0.6764
Epoch 8/50
125/125 [==============================] - 0s 938us/step - loss: 0.6566 - val_loss: 0.6554
Epoch 9/50
125/125 [==============================] - 0s 909us/step - loss: 0.6385 - val_loss: 0.6408
Epoch 10/50
125/125 [=============================

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 330.31it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9404 - val_loss: 0.9110
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8788 - val_loss: 0.8531
Epoch 3/50
125/125 [==============================] - 0s 925us/step - loss: 0.8253 - val_loss: 0.8027
Epoch 4/50
125/125 [==============================] - 0s 922us/step - loss: 0.7787 - val_loss: 0.7592
Epoch 5/50
125/125 [==============================] - 0s 930us/step - loss: 0.7388 - val_loss: 0.7219
Epoch 6/50
125/125 [==============================] - 0s 922us/step - loss: 0.7045 - val_loss: 0.6901
Epoch 7/50
125/125 [==============================] - 0s 934us/step - loss: 0.6758 - val_loss: 0.6636
Epoch 8/50
125/125 [==============================] - 0s 916us/step - loss: 0.6518 - val_loss: 0.6419
Epoch 9/50
125/125 [==============================] - 0s 911us/step - loss: 0.6335 - val_loss: 0.6268
Epoch 10/50
125/125 [==============================]